# Clase 4 - Segmentación de vídeo
![picture](https://drive.google.com/uc?id=1Ar6KpqlsLxcLlnOrXPx6QZejStP6bV9P)

## Pre requisitos

Se actualiza fastai descargando y ejecutando el script *colab*

In [0]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


Dar permiso para acceder a mi Drive

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Nos aseguramos que cualquier cambio en una librería es recargado automáticamente y que que cualquier gráfica o imagen se muestre aquí

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Importamos las librerías necesarias

In [0]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

## Clase 4 - Segmentación de vídeo

### Carga del modelo

Copiamos el modelo de Drive

In [0]:
drive_folder = '/content/drive/My\ Drive/Colab\ Notebooks/Curso\ fastai/04\ Segmentacion'
model_name = 'segmentacion.pkl'
model_tiramisu_name = 'segmentacion_tiramisu.pkl'
model_16b_name = 'segmentacion_16b.pkl'
model_tiramisu_16b_name = 'segmentacion_tiramisu_16b.pkl'

In [0]:
model_link = drive_folder + '/' + model_name
model_tiramisu_link = drive_folder + '/' + model_tiramisu_name
model_16b_link = drive_folder + '/' + model_16b_name
model_tiramisu_16b_link = drive_folder + '/' + model_tiramisu_16b_name

In [0]:
!cp {model_link} /content/data
!cp {model_tiramisu_link} /content/data
!cp {model_16b_link} /content/data
!cp {model_tiramisu_16b_link} /content/data

Definimos la función de métrica

In [0]:
def acc_camvid(input, target):
    target = target.squeeze(1)  # The output array is input array, but with all or a subset of the dimensions of length 1 removed
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

Lo cargamos

In [0]:
path_model = Path("/content/data/")
learn34 = load_learner(path_model, 'segmentacion.pkl')
learn34_tiramisu = load_learner(path_model, model_tiramisu_name)
learn34_16b = load_learner(path_model, model_16b_name).to_fp16()
learn34_tiramisu_16b = load_learner(path_model, model_tiramisu_16b_name).to_fp16()

In [0]:
path = untar_data(URLs.CAMVID)

### Función mask to RGB

In [0]:
codes = np.loadtxt(path/'codes.txt', dtype=str);
dict_codes = dict(enumerate(codes));

In [0]:
dict_colors = {
    "Animal":  [64, 128, 64],
    "Archway": [192, 0, 128],
    "Bicyclist": [0, 128, 192],
    "Bridge":  [0, 128, 64],
    "Building":  [128, 0, 0],
    "Car": [64, 0, 128],
    "CartLuggagePram": [64, 0, 192],
    "Child": [192, 128, 64],
    "Column_Pole": [192, 192, 128],
    "Fence": [64, 64, 128],
    "LaneMkgsDriv":  [128, 0, 192],
    "LaneMkgsNonDriv": [192, 0, 64],
    "Misc_Text": [128, 128, 64],
    "MotorcycleScooter": [192, 0, 192],
    "OtherMoving": [128, 64, 64],
    "ParkingBlock":  [64, 192, 128],
    "Pedestrian":  [64, 64, 0],
    "Road": [128, 64, 128],
    "RoadShoulder": [128, 128, 192],
    "Sidewalk":  [0, 0, 192],
    "SignSymbol":  [192, 128, 128],
    "Sky": [128, 128, 128],
    "SUVPickupTruck":  [64, 128, 192],
    "TrafficCone": [0, 0, 64],
    "TrafficLight":  [0, 64, 64],
    "Train": [192, 64, 128],
    "Tree":  [128, 128, 0],
    "Truck_Bus": [192, 128, 192],
    "Tunnel":  [64, 0, 64],
    "VegetationMisc":  [192, 192, 0],
    "Void":  [0, 0, 0],
    "Wall":  [64, 192, 0]
    }

In [0]:
def maskToRGB(mask, to='fastai'):

  # Se quita la dimensión que sea 1
  mask = mask.squeeze()
  
  # Se crea una matriz con el mismo tamaño que la máscara
  shape = mask.shape
  if to=='fastai':
    shape = (3, shape[0], shape[1])
  if to=='opencv':
    shape = (shape[0], shape[1], 3)
  RGB = np.empty(shape, dtype=np.uint8)
  if to=='fastai':
    RGB = torch.tensor(RGB)
  
  # Se convierte cada pixel de la máscara a RGB
  for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
      code = int(mask[i][j])
      color = dict_colors[dict_codes[code]]

      if to=='fastai':
        RGB[0][i][j] = color[0]/255
        RGB[1][i][j] = color[1]/255
        RGB[2][i][j] = color[2]/255
      if to=='opencv':
        RGB[i][j][0] = color[0]
        RGB[i][j][1] = color[1]
        RGB[i][j][2] = color[2]
  
  return RGB

### Procesado del video

#### Copiamos el video de Drive

In [0]:
!cp {drive_folder}/0016E5_cortado.mp4 /content/
!ls

0016E5_cortado.mp4  data  drive  models


#### Obtenemos el numero de frames del video

In [0]:
import cv2
import time
import numpy as np

In [0]:
# Se captura el video
input_source = "0016E5_cortado.mp4"
cap = cv2.VideoCapture(input_source)

In [0]:
# Se obtiene el número de frames
totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
totalFrames

1802

In [0]:
cap.release()

#### Función de segmentación de video

In [0]:
def segmentacion (input_file, output_file, width, height, learn):

  # Barra de progreso
  from ipywidgets import IntProgress
  from IPython.display import display

  # Se captura el video
  input_source = input_file
  cap = cv2.VideoCapture(input_source)

  # Parametros
  frameWidth = width
  frameHeight = height
  fps = 30

  # Video writer
  vid_writer = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc(*'MP4V'), fps, (frameWidth, frameHeight))

  # print progress bar
  f = IntProgress(description="Progress:", min=0, max=totalFrames) # instantiate the bar
  display(f) # display the bar

  # Tiempo de segmentacion
  segmentation_total_time = 0

  cont = 1
  while 1:
      t = time.time()

      # Se obtiene el frame del video
      hasFrame, frame = cap.read()
      if not hasFrame:
        cv2.waitKey()
        break

      # Se convierte el frame a una imagen de tipo fastai
      img_fastai = Image(pil2tensor(frame, dtype=np.float32).div_(255))

      # Se obtiene la máscara segmentada de la imagen
      start_time = time.time()
      predict_class, predict_idx, outputs = learn.predict(img_fastai)
      end_time = time.time()
      segmentation_time = end_time - start_time
      segmentation_total_time += segmentation_time

      # Se pasa la máscara a una imagen de tipo opencv
      rgb = maskToRGB(predict_idx, to='opencv')

      # Se escribe en el video de salida    
      vid_writer.write(rgb)

      # Barra de progreso
      f.value += 1
      cont += 1

  vid_writer.release()
  cap.release()

  return segmentation_total_time/totalFrames

#### Camvid

Lo procesamos con el modelo entrenado con el dataset Camvid

In [0]:
camvidWidth = 960
camvidHeight = 720

In [0]:
tiempo_medio_segmentacion_camvid = segmentacion ("0016E5_cortado.mp4", "0016E5_camvid.mp4", camvidWidth, camvidHeight, learn34)
print("tiempo medio segmentacion camvid: %s segundos" % tiempo_medio_segmentacion_camvid)

IntProgress(value=0, description='Progress:', max=1802)

tiempo medio segmentacion camvid: 1.3378960443257492e-06 segundos


Lo copiamos a Drive

In [0]:
!cp /content/0016E5_camvid.mp4 {drive_folder}

#### Camvid tiramisu

Lo procesamos con el modelo entrenado con el dataset Camvid tiramisu

In [0]:
camvidTiramisuWidth = frameWidth = 480
camvidTiramisuHeight = 360

In [0]:
tiempo_medio_segmentacion_camvid_tiramisu = segmentacion ("0016E5_cortado.mp4", "0016E5_camvid_tiramisu.mp4", camvidTiramisuWidth, camvidTiramisuHeight, learn34_tiramisu)
print("tiempo medio segmentacion camvid tiramisu: %s segundos" % tiempo_medio_segmentacion_camvid_tiramisu)

IntProgress(value=0, description='Progress:', max=1802)

tiempo medio segmentacion camvid tiramisu: 1.8957055501482728e-06 segundos


Lo copiamos a Drive

In [0]:
#!cp /content/0016E5_camvid_tiramisu.mp4 {drive_folder}
!cp /content/0016E5_camvid_tiramisu_prueba.mp4 {drive_folder}

#### Camvid 16 bits

Lo procesamos con el modelo entrenado con el dataset Camvid y el modelo bajado a 16 bits

In [0]:
tiempo_medio_segmentacion_camvid_16b = segmentacion ("0016E5_cortado.mp4", "0016E5_camvid_16b.mp4", camvidWidth, camvidHeight, learn34_16b)
print("tiempo medio segmentacion camvid 16 bits: %s segundos" % tiempo_medio_segmentacion_camvid_16b)

IntProgress(value=0, description='Progress:', max=1802)

tiempo medio segmentacion camvid 16 bits: 1.26909601040077e-06 segundos


Lo copiamos a Drive

In [0]:
!cp /content/0016E5_camvid_16b.mp4 {drive_folder}

#### Camvid tiramisu 16 bits

Lo procesamos con el modelo entrenado con el dataset Camvid tiramisu y el modelo bajado a 16 bits

In [0]:
tiempo_medio_segmentacion_camvid_tiramisu_16b = segmentacion ("0016E5_cortado.mp4", "0016E5_camvid_tiramisu_16b.mp4", camvidTiramisuWidth, camvidTiramisuHeight, learn34_tiramisu_16b)
print("tiempo medio segmentacion camvid tiramisu 16 bits: %s segundos" % tiempo_medio_segmentacion_camvid_tiramisu_16b)

IntProgress(value=0, description='Progress:', max=1802)

tiempo medio segmentacion camvid tiramisu 16 bits: 1.1625882655746002e-06 segundos


Lo copiamos a Drive

In [0]:
!cp /content/0016E5_camvid_tiramisu_16b.mp4 {drive_folder}

## Fin

Se puede ver una muestra de cómo quedarían los vídeos segmentados en el siguiente [vídeo de Youtube](https://youtu.be/k1uZ46LMaW0)

In [0]:
print('ok')